In [8]:
import os
import dash
from dash import dcc, html, Input, Output, State, no_update
import plotly.graph_objects as go
from IPython.display import HTML, IFrame
import numpy as np
import copy

class InteractiveApp:

    def __init__(self, host="0.0.0.0", port=8080, display_host="127.0.0.1"):
        self.host, self.port = host, port
        self.app = dash.Dash(__name__)
        self.app.layout = html.Div()
        self._demo_builders = {}
        self._current_demo = None
        self._server_thread = None

        if "DEEPNOTE_PROJECT_ID" in os.environ:
            deepnote_id = os.environ["DEEPNOTE_PROJECT_ID"]
            self.url = f"https://{deepnote_id}.deepnoteproject.com"
            self.is_deepnote = True
        else:
            self.url = f"http://{display_host}:{self.port}"
            self.is_deepnote = False


    def register_demo(self, name: str, builder):
        self._demo_builders[name] = builder

    def use_demo(self, name: str, *args, **kwargs):
        if name not in self._demo_builders:
            raise KeyError(f"No demo named '{name}' registered")
        if self._current_demo is not None:
            self.app._callback_list = []
        builder = self._demo_builders[name]
        builder({"app": self.app}, *args, **kwargs)
        self._current_demo = name

    def run(self):
        self.app.run(jupyter_mode="external", host=self.host, port=self.port, debug=False, use_reloader=False)

    def embed(self, height=500):
        if self.is_deepnote:
            height = "100%"
        else:
            height = f"{height}px"
        return HTML(
            f'<iframe src="{self.url}" '
            f'style="width:100%; height:{height}; border:none;"></iframe>'
        )

class bot2D():
    def __init__(self):
        # Configuration of 2D robot in world frame, x,y = coordinates in world frame, theta=orientation
        self.x = 0
        self.y = 0
        self.theta = 0
        self.states = []
        
    def reset(self):
        first_state = self.states[0]
        self.x, self.y, self.theta = first_state
        del self.states[:]
        self.states = [first_state]
        
    def get_state(self):
        """Return the current bicycle state. The state is in (x,y,theta) format"""
        return (self.x, self.y, self.theta)
    
    def set_state(self,x=0,y=0,theta=0):
        """Sets the model new state"""
        self.x = x
        self.y = y
        self.theta = theta
        self.states.append([x, y, theta])
        
class Bicycle(bot2D):
    """Implementation of the kinematic bicycle model (rear wheel model)
    length = distance (in meters) between the wheels
    gamma_max = maximum steering angle
    speed_max = maximum speed
    """

    def __init__(self, length=1, delta_max=np.pi/3, vel_max=5):
        super().__init__()
        
        # Length between the wheels
        self.length = length
        
        # Maximum velocity and steering angle
        self.delta_max = delta_max
        self.vel_max = vel_max
        
        # Initial control inputs
        self.velocity = 0
        self.delta = 0
        
    def update_control(self, v, delta):
        self.velocity = np.clip(v, -self.vel_max, self.vel_max)
        self.delta = np.clip(delta, -self.delta_max, self.delta_max)
        
    def drive(self, dt=0.01):
        """
        Update the bicycle's state.
        """        
        # Compute velocity using control input and current orientation
        v_x = self.velocity * np.cos(self.theta)
        v_y = self.velocity * np.sin(self.theta)
        
        # Compute next position
        next_x = self.x + v_x * dt
        next_y = self.y + v_y * dt
                
        # Compute angular velocity given control input (velocity and
        # steering angle)
        omega = self.velocity * np.tan(self.delta) / self.length
        
        # Compute next orientation
        next_theta = self.theta + omega * dt
        self.set_state(next_x, next_y, next_theta)
        
    def reset(self):
        super().reset()
        self.velocity = 0
        self.delta = 0

max_iterations = 180
max_vel = 5         # robot's maximum velocity
max_delta = np.pi/3 # robot's maximum steering angle
bike_length = 0.25  # create a bicycle with .25 m between the rear and front wheel
robot_start_x = -2.8 # robot's starting x position
robot_start_y = -2.8 # robot's starting y position
robot_start_theta = np.pi/6 # robot's starting orientation
time_step = 0.05

bot = Bicycle(bike_length, max_delta, max_vel)

app = InteractiveApp(port=8080)   # Dash kwargs
app.run()

def labeled_slider(id, label, min, max, step, value, marks):
    return html.Div(
        style={'display': 'flex', 'alignItems': 'center', 'marginBottom': '0px'},
        children=[
            html.Label(label, style={'width': '12%', 'marginRight': '5%'}),
            html.Div(
                dcc.Slider(
                    id=id,
                    min=min,
                    max=max,
                    step=step,
                    value=value,
                    marks=marks,
                    updatemode="drag",
                    tooltip={"placement": "bottom"},
                ),
                style={'flex': '1'}
            )
        ]
    )
    
def bicycle_viz(ctx, dt: float = 0.05, frames_max: int = 300):

    app = ctx["app"]
    bike = Bicycle()

    UIREV = "bicycle-demo"
    
    base_fig = go.Figure()
    base_fig.add_trace(
        go.Scatter(x=[bike.x], y=[bike.y], mode="lines", line=dict(width=2, color="blue"))
    )
    # Robot axes (trace 1 & 2)
    base_fig.add_trace(
        go.Scatter(x=[bike.x, bike.x + 1], y=[bike.y, bike.y], mode="lines", line=dict(width=4, color="red"), showlegend=False)
    )
    base_fig.add_trace(
        go.Scatter(x=[bike.x, bike.x], y=[bike.y, bike.y + 1], mode="lines", line=dict(width=4, color="green"), showlegend=False)
    )

    base_fig.update_layout(
        margin=dict(l=0, r=0, t=20, b=0),
        xaxis=dict(range=[-10, 10], scaleanchor="y", scaleratio=1, title="x"),
        yaxis=dict(range=[-10, 10], title="y"),
        showlegend=False,
        uirevision=UIREV,
    )

    app.layout = html.Div(
        style={"display": "flex", "flexDirection": "column", "height": "100vh"},
        children=[
            dcc.Graph(id="bicycle-graph", figure=base_fig, style={"flex": "1 1 auto"}),
            dcc.Interval(id="timer", interval=30, n_intervals=0, disabled=True),
            html.Div(
                style={"display": "flex", "gap": "2rem", "flexWrap": "wrap", "padding": "1rem 0"},
                children=[
                    labeled_slider("velocity", "Velocity (m/s)", -bike.vel_max, bike.vel_max, 0.1, 0.0, marks = {}),
                    labeled_slider(
                        "delta",
                        "Steering Δ (deg)",
                        -np.degrees(bike.delta_max),
                        np.degrees(bike.delta_max),
                        1,
                        0.0,
                        marks={
                            int(-np.degrees(bike.delta_max)): "-δmax",
                            0: "0°",
                            int(np.degrees(bike.delta_max)): "δmax",
                        },
                    ),
                    html.Div(id="frame-label", style={"alignSelf": "center", "fontFamily": "monospace"}),
                    html.Button("Play", id="play-btn", n_clicks=0),
                    html.Button("Pause", id="pause-btn", n_clicks=0),
                    html.Button("Reset", id="reset-btn", n_clicks=0),
                ],
            ),
        ],
    )

    @app.callback(
        Output("timer", "disabled"),
        Input("play-btn", "n_clicks"),
        Input("pause-btn", "n_clicks"),
        State("timer", "disabled"),
        prevent_initial_call=True,
    )
    def toggle_timer(play_clicks, pause_clicks, disabled):
        trig_id = dash.callback_context.triggered_id
        return trig_id != "play-btn"

    @app.callback(
        Output("velocity", "value"),
        Output("delta", "value"),
        Output("timer", "disabled", allow_duplicate=True),
        Output("timer",  "n_intervals"),
        Output("bicycle-graph", "figure", allow_duplicate=True),
        Output("frame-label", "children", allow_duplicate=True),
        Input("reset-btn", "n_clicks"),
        prevent_initial_call=True,
    )
    def reset_state(_):
        bike.reset()
        fresh_fig = copy.deepcopy(base_fig)
        fresh_fig.data[0].x, fresh_fig.data[0].y = [bike.x], [bike.y]
        fresh_fig.data[1].x, fresh_fig.data[1].y = [bike.x, bike.x + 1], [bike.y, bike.y]
        fresh_fig.data[2].x, fresh_fig.data[2].y = [bike.x, bike.x], [bike.y, bike.y + 1]
        return 0.0, 0.0, True, 0, fresh_fig, f"Frame: 0/{frames_max}"
        

    @app.callback(
        Output("bicycle-graph", "extendData"),
        Output("timer", "disabled", allow_duplicate=True),
        Input("timer", "n_intervals"),
        State("bicycle-graph", "figure"),
        State("velocity", "value"),
        State("delta", "value"),
        prevent_initial_call=True,
    )
    
    def step(frame_idx, fig, v, delta_deg):
        if frame_idx >= frames_max:
            return no_update, True
            
        bike.update_control(v, np.radians(delta_deg))
        bike.drive(dt=dt)

        R = np.array(
            [[np.cos(bike.theta), -np.sin(bike.theta)], [np.sin(bike.theta), np.cos(bike.theta)]]
        )
        x_axis = R @ np.array([1.0, 0.0])
        y_axis = R @ np.array([0.0, 1.0])

        patch = {
            "x": [
                [bike.x],
                [bike.x, bike.x + x_axis[0]],
                [bike.x, bike.x + y_axis[0]],
            ],
            "y": [
                [bike.y],
                [bike.y, bike.y + x_axis[1]],
                [bike.y, bike.y + y_axis[1]],
            ],
        }
        return (patch, [0, 1, 2], {"x": [300, 2, 2], "y": [300, 2, 2]}), no_update

    @app.callback(Output("frame-label", "children"), Input("timer", "n_intervals"))
    def update_label(n):
        return f"Frame: {min(n, frames_max)}/{frames_max}"

app.register_demo("Bicycle", bicycle_viz)
app.use_demo("Bicycle")

Dash app running on http://0.0.0.0:8080/
